In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


In [3]:
df = pd.read_csv('/kaggle/input/bdnews1k/News From 2023-12-16 To 2023-12-31.csv', usecols=['summary', 'article'])

In [4]:
df.dropna()

,summary,article
0,পৃথিবীর দিকে ধেয়ে আসছে একাধিক সৌরঝড়। বিজ্ঞানীর...,পৃথিবীর দিকে ধেয়ে আসছে একাধিক সৌরঝড়। বিজ্ঞানীর...
1,কোপ্রোপার্টি লিমিটেড তার বর্ষপূর্তির সাফল্য উদ...,কোপ্রোপার্টি লিমিটেড তার বর্ষপূর্তির সাফল্য উদ...
2,নাশতকার অভিযোগে পল্টন মডেল থানায় করা মামলায় বি...,নাশতকার অভিযোগে পল্টন মডেল থানায় করা মামলায় বি...
3,বাংলাদেশের প্রকৃতি ও জীববৈচিত্র সংরক্ষণে বিশেষ...,বাংলাদেশের প্রকৃতি ও জীববৈচিত্র সংরক্ষণে বিশেষ...
4,পাহাড়ে-সমতলে হত্যাকাণ্ড ও সন্ত্রাসী হামলার নি...,পাহাড়ে-সমতলে হত্যাকাণ্ড ও সন্ত্রাসী হামলার নি...
...,...,...
1737,পৌষের মধ্যভাগে এসেও নেই অতীতের মতো কনকনে শীতের...,পৌষের মধ্যভাগে এসেও নেই অতীতের মতো কনকনে শীতের...
1738,দিনাজপুরের এম আব্দুর রহিম মেডিকেল কলেজ হাসপাতা...,দিনাজপুরের এম আব্দুর রহিম মেডিকেল কলেজ হাসপাতা...
1739,মোংলা কাস্টমস হাউজের সাবেক রাজস্ব কর্মকর্তা মো...,মোংলা কাস্টমস হাউজের সাবেক রাজস্ব কর্মকর্তা মো...
1740,দ্বাদশ জাতীয় সংসদ নির্বাচনে নওগাঁ-২ (পত্নীতলা ...,দ্বাদশ জাতীয় সংসদ নির্বাচনে নওগাঁ-২ (পত্নীতলা ...


In [5]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.30, shuffle=True)


In [6]:
from datasets import Dataset
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

In [21]:
ds_train['summary'][50]

'পাকিস্তানের সাবেক প্রধানমন্ত্রী ইমরান খান দেশটির আগামী নির্বাচনে তিনটি আসন থেকে প্রতিদ্বন্দ্বিতা করবেন। বর্তমানে তিনি আদিয়ালা জেলে বন্দী। বুধবার এ কথা জানিয়েছেন পিটিআই দলের আইনজীবি। খবর ডনের।'

In [7]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"

tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [8]:
def tokenize_data(data):
    input_features = tokenizer(data['article'], truncation=True, max_length=1024)
    label = tokenizer(data['summary'],truncation =True, max_length=128)
    return {
        'input_ids':input_features['input_ids'],
        'attention_mask': input_features['attention_mask'],
        'labels': label['input_ids'],
    }


In [9]:
tokenized_ds_train = ds_train.map(
    tokenize_data,
    batched=True,
    remove_columns=ds_train.column_names  # Remove original columns
)

# Apply tokenization to the test dataset
tokenized_ds_test = ds_test.map(
    tokenize_data,
    batched=True,
    remove_columns=ds_test.column_names
)

Map:   0%|          | 0/1219 [00:00<?, ? examples/s]

Map:   0%|          | 0/523 [00:00<?, ? examples/s]

In [10]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer = tokenizer,
    model = model,
    return_tensors = 'pt'
)

In [11]:
from transformers import Seq2SeqTrainingArguments

model.gradient_checkpointing_enable()

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps = 8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    fp16=True,
    predict_with_generate=False,
    logging_dir='./logs',
    logging_steps=10,
    generation_max_length = 128,
    push_to_hub = False,
    dataloader_num_workers=1,
)

In [12]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss
0,1.505300,1.325428


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disab

TrainOutput(global_step=76, training_loss=1.7279450579693443, metrics={'train_runtime': 458.9332, 'train_samples_per_second': 2.656, 'train_steps_per_second': 0.166, 'total_flos': 1765313433698304.0, 'train_loss': 1.7279450579693443, 'epoch': 0.9967213114754099})

In [14]:
pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=460bda193ca4f89da30db9d4b69f6ce2a71a1085d7e264f31454137233caa75c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


# Evaluate ROUGE scores

In [17]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
    encoded_arg = tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
    preds, labels = eval_arg

    # Replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Convert id tokens to text

    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring

    # (Note : Please change this code, when you perform on other languages except for Bengali)
    text_preds = [(p if p.endswith(("!", "!", "?", "?", "।")) else p + "।") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "!", "?", "?", "।")) else l + "।") for l in text_labels]
    sent_tokenizer_bn = RegexpTokenizer(u'[^!!??।]*[!!??।]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(l))) for l in text_labels]

    # compute ROUGE score with custom tokenization
    return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
    )

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [21]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
    tokenized_ds_test.with_format("torch"),
    collate_fn=data_collator,
    batch_size=1
)

for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=2,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_length=128
        )
    labels = batch["labels"]
    break

metrics_func([preds, labels])

{'rouge1': 0.4712041884816754,
 'rouge2': 0.2962962962962963,
 'rougeL': 0.4607329842931937,
 'rougeLsum': 0.47715736040609136}